![Insper](https://github.com/danielscarvalho/Insper-DS-Dicas/blob/master/Insper-Logo.png?raw=true)

# Insper Pós-Graduação
## Programa Avançado em Data Science e Decisão [»](https://www.insper.edu.br/pos-graduacao/programas-avancados/programa-avancado-em-data-science-e-decisao/)


# Atividade Integradora
## Setup

### Dependências

In [ ]:
import pandas as pd
import numpy as np
from dfply import *
import altair as alt
import missingno as msno
from ydata_profiling import ProfileReport
import matplotlib
import matplotlib.pyplot as plt 
import math
import seaborn as sns
from sklearn import linear_model
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import statsmodels.api as sm

### Carregamento dos Dados

Leitura da base de dados e do dicionários de dados

In [ ]:
data = pd.read_csv("cs_bisnode_panel.csv")
dicionario_de_dados_0 = pd.read_excel("bisnode_variable_names.xls", header=4)

Visualização inicial da base de dados

In [ ]:
data.head()

In [ ]:
data.describe()

## Dicionário de dados
### Limpeza

Ao carregar o dicionário de dados a primeira coluna pega seu nome da quarta linha da tabela (argumento `header=4` acima). As outras três colunas são nomeadas abaixo.

In [ ]:
dicionario_de_dados_1 = dicionario_de_dados_0.rename({'Unnamed: 1': 'description',
                                                      'Unnamed: 2': 'type',
                                                      'Unnamed: 3': 'footnote'},
                                                     axis=1)

In [ ]:
dicionario_de_dados_1.columns

Então, retiramos as linhas não relevantes para a análise, incluindo linhas totalmente em branco e uma linha com informação de versão da base de dados: 
 - `v 0.92. 2021-02-04`

In [ ]:
dicionario_de_dados = dicionario_de_dados_1\
                       .drop(index=54)\
                       .dropna(how="all")\
                       .reset_index()\
                       .drop('index', axis='columns')

In [ ]:
dicionario_de_dados.sample(5)

## Dados faltantes
---

###  Visualização
Vamos verificar dados faltando do banco de dados:

Criação de função para analisar os dados faltantes.

In [ ]:
def show_missing(df):
    """Return a Pandas dataframe describing the contents of a source dataframe including missing values."""
    
    variables = []
    dtypes = []
    count = []
    unique = []
    missing = []
    pc_missing = []
    
    for item in df.columns:
        variables.append(item)
        dtypes.append(df[item].dtype)
        count.append(len(df[item]))
        unique.append(len(df[item].unique()))
        missing.append(df[item].isna().sum())
        pc_missing.append(round((df[item].isna().sum() / len(df[item])) * 100, 2))

    output = pd.DataFrame({
        'variable': variables, 
        'dtype': dtypes,
        'count': count,
        'unique': unique,
        'missing': missing, 
        'pc_missing': pc_missing
    })    
        
    return output

Verificando dados com maior falta de informações:

In [ ]:
missing_data = show_missing(data).sort_values('pc_missing', ascending=False, ignore_index = True)

index_full_data = list(missing_data[missing_data['missing']==0].index)

missing_data.drop(labels=index_full_data, axis='index', inplace=True)

missing_data.head(10)

In [ ]:
missing_val_columns = missing_data['variable'][missing_data['pc_missing']>0]

In [ ]:
msno.bar(data[missing_val_columns], figsize=(16, 4))

### Colunas com poucos dados
Removemos aqui seis colunas tem mais de 90% de dados faltantes, removemos as colunas "begin" e "end" que contêm informação sobre o período a que os dados dizem respeito pois consideramos na análise essa mesma informação contida na variável "year". Removemos também as variáveis 'exit_year' e 'exit_data', visto que o encerrameto da empresa é definido com base na variável 'sales'.

In [ ]:
columns_to_remove = ['COGS',
                     'finished_prod',
                     'net_dom_sales',
                     'net_exp_sales',
                     'wages',
                     'D',
                     'begin',
                     'end',
                     'exit_year',
                     'exit_date'] 

data.drop(columns=columns_to_remove, inplace=True)

data.columns

### "founded_year"
Aproveitamos para tratar aqui a variável "founded_year" que utilizamos mais à frente na criação da feature "company_age".

In [ ]:
missing_data.loc[missing_data['variable'].isin(['founded_year','founded_date'])]

Convertemos 'founde_date' para datetime.

In [ ]:
data.dtypes['founded_date']

In [ ]:
data['founded_date'] = pd.to_datetime(data['founded_date'], format='%Y-%m-%d')

data['founded_date'].dtypes

Utilizamos a variável "founded_date" (que tem uma menor proporção de valores faltantes) para preencher alguns dos valores faltantes em "founded_year" (que tem a granularidade desejada para a análise).

In [ ]:
assert any(data.loc[(data['founded_date'].isna() & 
                     data['founded_year'].isna()),
                    'comp_id'].count() ==\
           missing_data.loc[missing_data['variable'] == 'founded_date', 'missing'])

As observações sem informação em "founded_date" também não tem informação para "founded_year" e são retiradas.

In [ ]:
# Remove linhas onde 'founded_date' é NaN
data = data.dropna(subset=['founded_date'])

# Extrai o ano de 'founded_date' e substitui os valores ausentes em 'founded_year'
data['founded_year'] = data['founded_date'].dt.year

# Descarta a coluna 'founded_date', pois o ano de fundação da empresa é suficiente
data = data.drop(columns=['founded_date'])

### Checkpoint
Análise dos percentuais de missing após os tratamentos.

In [ ]:
missing_data2 = show_missing(data).sort_values('pc_missing', ascending=False, ignore_index = True)

index_full_data = list(missing_data2[missing_data2['missing']==0].index)

missing_data2.drop(labels=index_full_data, axis='index', inplace=True)

missing_data2

### Colunas com dados quase completos

Aqui tratamos as colunas com 5% ou menos dados faltantes. Primeiramente checamos se esses dados estão faltando nas mesmas observações ou se estão distribuidos em observações diferentes.

In [ ]:
# Lista de colunas com menos de 5% de dados ausentes

cols = list(missing_data2['variable'][missing_data2['pc_missing']<5.0])

# Verifica quais linhas possuem dados ausentes nessas colunas

missing_rows = data[cols].isnull().any(axis=1)      

# Calcula o percentual de linhas com dados ausentes

pc_missing_rows = 100 * missing_rows.sum() / len(data)

print(f'Percentual de linhas com dados ausentes em pelo menos uma das colunas mencionadas: {pc_missing_rows:.2f}%')

In [ ]:
# Remove variáveis com menos de 5% de dados ausentes
removable_na_columns = list(missing_data2['variable'][missing_data2['pc_missing']<5.0])

data.dropna(subset=removable_na_columns, inplace=True)

Valores faltantes após os tratamentos.

In [ ]:
missing_data_3 = show_missing(data).sort_values('pc_missing', ascending=False, ignore_index = True)

index_full_data = list(missing_data_3[missing_data_3['missing']==0].index)

missing_data_3.drop(labels=index_full_data, axis='index', inplace=True)

missing_data_3

In [ ]:
msno.matrix(data)

A tabela mostra 6 variáveis com cerca de 20% de dados faltantes nesse ponto da análise, e o gráfico indica que essa ausência de dados coincide nas mesmas observações. Optamos por deixar essas observações apesar da coincidência de dados faltantes, e tratar as faltas mais abaixo.

## Tratamento de demais variáveis com MICE

Os demais valores ausentes são preenchidos utilizando o método MICE (Multiple Imputation by Chained Equations) com a biblioteca 'IterativeImputer' do sklearn. 



In [ ]:
data.reindex()

Antes de proceder com a imputação usando o método MICE, é importante converter certas variáveis para o tipo categórico. Isso garante que essas variáveis sejam tratadas de maneira adequada durante o processo de imputação.

In [ ]:
# Lista das colunas para transformar em categóricas
colunas_para_converter = ['balsheet_flag', 'balsheet_notfullyear', 'gender', 'origin', 'ind2', 'ind', 'urban_m', 'region_m']

# Convertendo as colunas para o tipo 'category'
data[colunas_para_converter] = data[colunas_para_converter].astype('category')

# Verificando os tipos das colunas após a conversão
print(data[colunas_para_converter].dtypes)


In [ ]:
df_mice = data.copy()
df_mice.head()

In [ ]:
# Copia o df_mice para criar variáveis dummy
df_mice2 = df_mice.copy()
df_mice2 = pd.get_dummies(df_mice2)

In [ ]:
# Define Input de MICE e preenche NaN 

mice_imputer = IterativeImputer(estimator=linear_model.BayesianRidge(), n_nearest_features=None, imputation_order='ascending')
df_mice_imputed = pd.DataFrame(mice_imputer.fit_transform(df_mice2), columns=df_mice2.columns)

In [ ]:
df_mice_imputed.head()

In [ ]:
missing_data_3 = show_missing(df_mice_imputed).sort_values("pc_missing", ascending=False, ignore_index = True)

missing_data_3

In [ ]:
# Verifica dados imputados 

# MICE imputação
fig = plt.Figure()
null_values = data['labor_avg'].isnull() 
fig = df_mice_imputed.plot(x='sales', y='labor_avg', kind='scatter',
                           c=null_values, cmap='winter', s = 15,
                           title='MICE Imputation', colorbar=False)

In [ ]:
df_mice_imputed.columns

Reverte o processo de pd.get_dummies com base no df_mice_imputed

In [ ]:
missing_data_4 = show_missing(df_mice_imputed).sort_values("pc_missing", ascending=False, ignore_index = True)

missing_data_4.sort_values(by="variable", ignore_index=True)

## Feature Engineering
---

### Variável Dependente - Inatividade da Empresa
Chamamos "inativas" aquelas empresas que não apresentam vendas nos dois anos seguintes ao ano observado.\
**Nota:** ativa = 0, inativa = 1

In [ ]:
# Ordena o DataFrame por empresa e ano
df_mice_imputed.sort_values(by=['comp_id', 'year'], inplace=True)

# Cria colunas deslocadas para checar vendas nos dois anos seguintes
conditions_x1 = [((df_mice_imputed['comp_id'] == df_mice_imputed['comp_id'].shift(-1)) &
                  (df_mice_imputed['year'] == df_mice_imputed['year'].shift(-1) - 1)),

                 ((df_mice_imputed['comp_id'] != df_mice_imputed['comp_id'].shift(-1)) |
                  df_mice_imputed['year'] != df_mice_imputed['year'].shift(-1))]

values_x1 = [df_mice_imputed['sales'].shift(-1),
             np.nan]

df_mice_imputed['sales_x1'] = pd.Series(np.select(conditions_x1, values_x1)).fillna(0)


conditions_x2 = [
    ((df_mice_imputed['comp_id'] == df_mice_imputed['comp_id'].shift(-1)) &   
     (df_mice_imputed['year'] == df_mice_imputed['year'].shift(-1) - 2)),

    ((df_mice_imputed['comp_id'] == df_mice_imputed['comp_id'].shift(-2)) &
     (df_mice_imputed['year'] == df_mice_imputed['year'].shift(-2) - 2)),

    True
]

values_x2 = [df_mice_imputed['sales'].shift(-1),
             df_mice_imputed['sales'].shift(-2),
             np.nan]

df_mice_imputed['sales_x2'] = pd.Series(np.select(conditions_x2, values_x2)).fillna(0)


# Define condicao usada para identificar empresas que pararam de operar:
#  - sem vendas por mais de 2 anos
condition = ((df_mice_imputed['sales_x1'] == 0) & (df_mice_imputed['sales_x2'] == 0)) 

# Create a new 'dependente' column with 1 for ceased companies and 0 otherwise
df_mice_imputed['fechado'] = condition.astype(int)

Vamos conferir os valores da variavel dependente para algumas empresas do dataframe:

In [ ]:
# lista de colunas para avaliação
check_list = ['comp_id', 'year', 'sales', 'sales_x1', 'sales_x2', 'fechado']

filtro = df_mice_imputed[check_list]
filtro_sub = filtro[(filtro['comp_id'] == 464021159936) | (filtro['comp_id'] == 1001541)]

filtro_sub

In [ ]:
# Remodela os dados de vendas para tratar valores negativos
df_mice_imputed['sales'] = np.where(df_mice_imputed['sales']<0, 0, df_mice_imputed['sales'])

df_mice_imputed['fechado'] = condition.astype(int)

In [ ]:
df_mice_imputed[check_list].describe()

Vamos averiguar agora os anos em que as empresas tiveram atividade e inatividade:

In [ ]:
# Converte "comp_id" para float antes de agrupar
df_mice_imputed['comp_id'] = df_mice_imputed['comp_id'].astype(float)

data_grouped = df_mice_imputed.groupby('comp_id')

# Conta anos de acompanhamento
comp_years = data_grouped['year'].count()

# Conta anos com vendas
sales_years = data_grouped.apply(lambda group: (group['sales'] > 0).sum())

# Conta anos sem vendas
no_sales_years = data_grouped.apply(lambda group: (group['sales'] == 0).sum())

# Conta anos de "inatividade"
inative_years = data_grouped.apply(lambda group: (group['fechado'] == 1).sum())

In [ ]:
pd.options.display.float_format = '{:.1f}'.format

activity_df = pd.DataFrame({'Total years':comp_years,
                            'Sales years':sales_years, 
                            'No sales years':no_sales_years,
                            'Inative years':inative_years}).reset_index()

activity_df['comp_id'] = activity_df['comp_id'].astype(float)
activity_df.head()

## Seleção de subset para análise
---
A modelagem é feita em cima de um subset dos dados com o objetivo de adicionar foco à analise e facilitar o processamento por estar trabalhando com menor volume de dados. O escopo desse subset pode ser adaptado iterativamente, atendendo a demandas que surjam na análise.

### Removendo dados do ano 2016
Registros do ano de 2016 são removidos do conjunto.

In [ ]:
df_mice_imputed = df_mice_imputed[df_mice_imputed['year']!=2016].copy()
assert not any(df_mice_imputed["year"] == 2016)

### Ano

In [ ]:
# Cria dataframe apenas com os dados do ano de 2012
data_2012 = df_mice_imputed[df_mice_imputed['year'] == 2012].copy()

assert data_2012.year.unique() == 2012 # verificando coluna de ano

### Tamanho da Empresa

In [ ]:
# Considera apenas sales > 1000 e < 10000000
data_2012 = data_2012[(data_2012['sales'] > 1000) & (data_2012['sales'] < 10000000) ]

Verificando colunas com valores negativos

In [ ]:
# Identificando colunas com valores númericos no data_2012
numeric_cols_2012 = data_2012.select_dtypes(include=[np.number]).columns

# Creando dicionário para armazenar colunas com valores negativos 
negative_values_2012 = {}

# Iteração nas colunas númericas para encontrar valores negativos
for col in numeric_cols_2012:
    negative_count = (data_2012[col] < 0).sum()
    if negative_count > 0:
        negative_values_2012[col] = negative_count

# Mostrar colunas com valores negativos e contagem de valores negativos
negative_values_2012

Analisando as colunas com valores negativos, é constatado que faz sentido ter valores negativos nas colunas 'extra_profit_loss',  'inc_bef_tax', 'profit_loss_year' e  'share_eq'. Enquanto nas demais colunas, não faz sentido.

In [ ]:
# Lista das colunas para verificar valores negativos, excluindo as colunas que fazem sentido ter valores negativos
columns_to_check = [
    'amort', 'curr_assets', 'curr_liab', 'extra_exp', 'extra_inc', 
    'intang_assets', 'inventories', 'liq_assets', 'material_exp', 
    'personnel_exp', 'subscribed_cap', 'labor_avg', 
    'sales_x1', 'sales_x2'
]

# Identificar as colunas que possuem valores negativos 
negative_rows = [data_2012[data_2012[col] < 0].index for col in columns_to_check]

# Começar índice de valores negativos na primeira coluna 
all_negative_rows = set(negative_rows[0])

# Unir os índices de valores negativos de todas as colunas
for row_set in negative_rows[1:]:
    all_negative_rows = all_negative_rows.union(row_set)

# Número de linhas com qualquer valor negativo dessas colunas selecionadas 
len_all_negative_rows = len(all_negative_rows)

# Calcular percentual de linhas com valores negativos
percentage_any_negative = (len_all_negative_rows / len(data_2012)) * 100

# Formatar o percentual e mostrar valor
formatted_percentage_any_negative = "{:.2f}%".format(percentage_any_negative)
print(formatted_percentage_any_negative)


In [ ]:
# Para cada linha do index com valores negativos 
for row_idx in all_negative_rows:
    # Para cada coluna específica
    for col in columns_to_check:
        # Se o valor for negativo, considere como igual a 0
        if data_2012.at[row_idx, col] < 0:
            data_2012.at[row_idx, col] = 0

# Verificando se ainda tem valores negativos após o tratamento 
negative_values_after_correction = {col: data_2012[data_2012[col] < 0].shape[0] for col in columns_to_check}
negative_values_after_correction

## Feature Engineering no Subset

### Assimetria
Nessa subsessão olhamos as medidas de assimetria das variáveis na nossa base. Para a presente análise tratamos apenas a variável "sales", criando outra variável a partir de seu logarítmo. A análise das demais variáveis é disponibilizada para interpretação e trabalho subsequente.

In [ ]:
# Filtra apenas colunas numéricas para calcular assimetria
numeric_cols = data_2012.select_dtypes(include=[np.number]).columns # Avaliar a simetria das variáveis

# Avalia a simetria das colunas numéricas
skewness_values = data_2012[numeric_cols].skew()

# Ordena os valores de assimetria do maior para o menor
sorted_skewness_values = skewness_values.sort_values(ascending=False)

# Mostra os valores de assimetria ordenados
print(sorted_skewness_values)

In [ ]:
# Identifica colunas com assimetria significativamente diferente de zero
cols_to_transform = skewness_values[skewness_values.abs() > 0.5].index
print(cols_to_transform)

Nas estatísticas descritivas abaixo, observa-se que a média é maior que o terceiro quartil, indicando uma distribuição bastante assimétrica. Criamos então uma coluna com o logarítmo de "sales" para auxiliar na análise.

In [ ]:
print(data_2012['sales'].describe())

A coluna log_sales mostra que, embora muitas empresas tenham vendas relativamente baixas (indicado por um logaritmo de vendas próximo ou igual a zero), há empresas que se destacam com vendas substancialmente mais altas.

In [ ]:
# Função para criar coluna com logaritmo da coluna argumento
def criar_log(data_set, col):
    new_col = 'log_' + col
    data_set.loc[:,new_col] = data_set.loc[:,col]\
                                            .apply(lambda x: math.log(x)\
                                                   if x > 0\
                                                   else 0) 

In [ ]:
criar_log(data_2012, 'sales')
data_2012.log_sales.describe()

A descrição acima mostra valores mais simétricos para o logarítmo de "sales", quando comparado com "sales".

#### "sales" vs "log_sales"

In [ ]:
# Cria histogramas
plt.figure(figsize=(12, 6))

# Histograma de sales
plt.subplot(1, 2, 1)
plt.hist(data_2012['sales'], bins=50, color='blue', edgecolor='black')
plt.title('Histograma de Sales')
plt.xlabel('Sales')
plt.ylabel('Frequência')

# Histograma de log_sales
plt.subplot(1, 2, 2)
plt.hist(data_2012['log_sales'], bins=50, color='green', edgecolor='black')
plt.title('Histograma de Log de Sales')
plt.xlabel('Log Sales')
plt.ylabel('Frequência')

plt.tight_layout()
plt.show()


#### Idade da Empresa

In [ ]:
data_2012['company_age'] = 2012 - data_2012['founded_year']

#### Outras Features

A fim de aprimorar a precisão e eficácia de nosso modelo, estamos considerando a introdução de novas variáveis explanatórias. A motivação por trás dessa decisão é explorar a possibilidade de que certas variáveis, ainda não incluídas em nosso modelo, possam ter um impacto significativo no erro do modelo. Ao identificar e incorporar essas variáveis potencialmente influentes, esperamos não apenas reduzir o erro, mas também aumentar a robustez do modelo, tornando suas previsões mais confiáveis e precisas. Nos próximos passos, avaliaremos cada uma dessas variáveis, sua relevância e a maneira como elas podem ser integradas ao modelo existente.

In [ ]:
# Definir função para que, se o denominador for igual a 0, retornar zero
def safe_divide(numerator, denominator, default_value=0.0):
    if denominator == 0:
        return default_value
    return numerator / denominator

# Ajustar as novas variáveis com safe division 
data_2012["current_ratio"] = data_2012.apply(lambda row: safe_divide(row["curr_assets"], row["curr_liab"]), axis=1)
data_2012["profit_margin"] = data_2012.apply(lambda row: safe_divide(row["profit_loss_year"], row["sales"]), axis=1)
data_2012["debt_equity_ratio"] = data_2012.apply(lambda row: safe_divide((row["liq_assets"] - row["curr_liab"]), row["share_eq"]), axis=1)
data_2012["asset_turnover"] = data_2012.apply(lambda row: safe_divide(row["sales"], (row["curr_assets"] + row["fixed_assets"])), axis=1)

# Exibindo as primeiras linhas do df atualizado com as novas variáveis
data_2012[["current_ratio", "profit_margin", "debt_equity_ratio", "asset_turnover"]].head()

### Liquidez

A razão corrente (current_ratio), um indicador de liquidez, mede a capacidade de uma empresa de pagar suas obrigações de curto prazo usando ativos de curto prazo. Empresas com baixa liquidez podem ter dificuldades em cumprir suas obrigações financeiras imediatas, o que pode levar a problemas financeiros e aumentar o risco de encerramento no curto prazo.

In [ ]:
# Configurando as propriedades do gráfico
plt.figure(figsize=(10, 6))

# Histograma para empresas que não fecharam
plt.hist(data_2012["current_ratio"][data_2012['fechado'] == 0], bins=50, color='blue', alpha=0.7, label='Fechado = 0', density=True)

# Histograma para empresas que fecharam
plt.hist(data_2012["current_ratio"][data_2012['fechado'] == 1], bins=50, color='red', alpha=0.7, label='Fechado = 1', density=True)

# Definindo o título e os labels dos eixos
plt.title('Distribuição da Liquidez por Status da Empresa')
plt.xlabel('Liquidez (current_ratio)')
plt.ylabel('Densidade') # Mostrar densidade ao invés da contagem, dado a diferença entre a qtd de empresas fechadas e abertas
plt.legend()

# Definindo o limite para o eixo x 
plt.xlim(-1000, 1000)

# Mostrando o gráfico
plt.show()

In [ ]:
# Separando os dados por grupo
group_0 = data_2012["current_ratio"][data_2012['fechado'] == 0]
group_1 = data_2012["current_ratio"][data_2012['fechado'] == 1]

# Calculando as estatísticas
stats_0 = {
    'mean': group_0.mean(),
    'median': group_0.median(),
    'std_dev': group_0.std(),
    'min': group_0.min(),
    'max': group_0.max(),
    '25_percentile': group_0.quantile(0.25),
    '50_percentile': group_0.quantile(0.50),
    '75_percentile': group_0.quantile(0.75),
    'iqr': group_0.quantile(0.75) - group_0.quantile(0.25),
    'mode': group_0.mode().iloc[0],  # Pega o primeiro valor da moda
    'skewness': group_0.skew(),
    'kurtosis': group_0.kurt(),
    'sample_size': len(group_0)
}

stats_1 = {
    'mean': group_1.mean(),
    'median': group_1.median(),
    'std_dev': group_1.std(),
    'min': group_1.min(),
    'max': group_1.max(),
    '25_percentile': group_1.quantile(0.25),
    '50_percentile': group_1.quantile(0.50),
    '75_percentile': group_1.quantile(0.75),
    'iqr': group_1.quantile(0.75) - group_1.quantile(0.25),
    'mode': group_1.mode().iloc[0],  # Pega o primeiro valor da moda
    'skewness': group_1.skew(),
    'kurtosis': group_1.kurt(),
    'sample_size': len(group_1)
}

# Criando DataFrames para exibir as estatísticas
df_stats_0 = pd.DataFrame(stats_0, index=['Fechado = 0'])
df_stats_1 = pd.DataFrame(stats_1, index=['Fechado = 1'])

# Concatenando os DataFrames
df_stats_current_ratio = pd.concat([df_stats_0, df_stats_1])

df_stats_current_ratio

Conforme evidenciado no histograma e nas estatísticas descritivas, destaca-se os pontos abaixo:
- A liquidez média das empresas que não fecharam é ligeiramente superior às que fecharam. 
- A alta assimetria e curtose sugerem que existem algumas empresas com liquidez extremamente alta, o que está distorcendo a média.
- O alto valor da moda em 0 indica que muitas empresas, em ambos os grupos, têm uma liquidez de zero.

### Margem de lucro

A Margem de Lucro mede a porcentagem de lucro que uma empresa obtém de suas vendas. Empresas com margens de lucro baixas podem ter dificuldades em absorver choques financeiros ou econômicos. Uma margem de lucro consistentemente baixa pode indicar problemas operacionais ou de eficiência, aumentando o risco de a empresa fechar no futuro.

In [ ]:
# Criar o violin plot da margem de lucro, separado por 'fechado'
plt.figure(figsize=(10, 6))
sns.violinplot(x="fechado", y="profit_margin", data=data_2012, palette={0: 'blue', 1: 'red'}, inner="quartile")

# Definir o título e os labels dos eixos
plt.title('Margem de Lucro vs Fechado')
plt.xlabel('Fechado')
plt.ylabel('Margem de Lucro')
plt.ylim(-50, 50)  # Definindo o limite para o eixo y

# Mostar o gráfico
plt.show()

In [ ]:
# Separar os dados por grupo e por profit_margin
group_0_profit_margin = data_2012["profit_margin"][data_2012['fechado'] == 0]
group_1_profit_margin = data_2012["profit_margin"][data_2012['fechado'] == 1]

# Calcular as estatísticas
stats_0_profit_margin = {
    'mean': group_0_profit_margin.mean(),
    'median': group_0_profit_margin.median(),
    'std_dev': group_0_profit_margin.std(),
    'min': group_0_profit_margin.min(),
    'max': group_0_profit_margin.max(),
    '25_percentile': group_0_profit_margin.quantile(0.25),
    '50_percentile': group_0_profit_margin.quantile(0.50),
    '75_percentile': group_0_profit_margin.quantile(0.75),
    'iqr': group_0_profit_margin.quantile(0.75) - group_0_profit_margin.quantile(0.25),
    'mode': group_0_profit_margin.mode().iloc[0],  
    'skewness': group_0_profit_margin.skew(),
    'kurtosis': group_0_profit_margin.kurt(),
    'sample_size': len(group_0_profit_margin)
}

stats_1_profit_margin = {
    'mean': group_1_profit_margin.mean(),
    'median': group_1_profit_margin.median(),
    'std_dev': group_1_profit_margin.std(),
    'min': group_1_profit_margin.min(),
    'max': group_1_profit_margin.max(),
    '25_percentile': group_1_profit_margin.quantile(0.25),
    '50_percentile': group_1_profit_margin.quantile(0.50),
    '75_percentile': group_1_profit_margin.quantile(0.75),
    'iqr': group_1_profit_margin.quantile(0.75) - group_1_profit_margin.quantile(0.25),
    'mode': group_1_profit_margin.mode().iloc[0],  
    'skewness': group_1_profit_margin.skew(),
    'kurtosis': group_1_profit_margin.kurt(),
    'sample_size': len(group_1_profit_margin)
}

# Criar dataframe para exibir profit_margin
df_stats_0_profit_margin = pd.DataFrame(stats_0_profit_margin, index=['Fechado = 0'])
df_stats_1_profit_margin = pd.DataFrame(stats_1_profit_margin, index=['Fechado = 1'])

# Concatenar DataFrames por profit_margin
df_stats_profit_margin_final = pd.concat([df_stats_0_profit_margin, df_stats_1_profit_margin])

df_stats_profit_margin_final

Conforme observado no gráfico e nas estatística descritivas, podemos inferir que a margem de lucro é uma variável importante a ser considerada ao analisar a sobrevivência de uma empresa. A margem de lucro média é negativa para ambas as categorias de empresas, mas as empresas que fecharam têm uma margem de lucro ligeiramente mais negativa em média. No entanto, a variabilidade nas margens de lucro é maior entre as empresas que permaneceram abertas. Isso sugere que, enquanto algumas empresas que permaneceram abertas podem estar enfrentando desafios financeiros, outras estão prosperando e gerando altos lucros.

### Relação Dívida/Patrimônio Líquido

A Relação Dívida/Patrimônio Líquido indica quanto de financiamento a empresa obteve por dívida em relação ao seu patrimônio líquido. Uma alta relação pode sugerir que a empresa está excessivamente endividada, o que pode aumentar o risco financeiro. Empresas altamente endividadas podem enfrentar desafios em cumprir suas obrigações, especialmente em tempos de instabilidade econômica, elevando o risco de encerramento.

In [ ]:
plt.figure(figsize=(10, 6))

sns.violinplot(x="fechado", y="debt_equity_ratio", data=data_2012, palette="Set3", inner="quartile")

plt.title('Violinplot de Dívida/patrimônio líquido separado por Fechado')
plt.xlabel('Fechado')
plt.ylabel('Dívida/patrimônio líquido')
plt.ylim(-500, 500)  # Definindo o limite para o eixo y

plt.show()

In [ ]:
# Separar os dados por grupo
group_0 = data_2012["debt_equity_ratio"][data_2012['fechado'] == 0]
group_1 = data_2012["debt_equity_ratio"][data_2012['fechado'] == 1]

# Calcular as estatísticas
stats_0 = {
    'mean': group_0.mean(),
    'median': group_0.median(),
    'std_dev': group_0.std(),
    'min': group_0.min(),
    'max': group_0.max(),
    '25_percentile': group_0.quantile(0.25),
    '50_percentile': group_0.quantile(0.50),
    '75_percentile': group_0.quantile(0.75),
    'iqr': group_0.quantile(0.75) - group_0.quantile(0.25),
    'mode': group_0.mode().iloc[0],  # Pega o primeiro valor da moda
    'skewness': group_0.skew(),
    'kurtosis': group_0.kurt(),
    'sample_size': len(group_0)
}

stats_1 = {
    'mean': group_1.mean(),
    'median': group_1.median(),
    'std_dev': group_1.std(),
    'min': group_1.min(),
    'max': group_1.max(),
    '25_percentile': group_1.quantile(0.25),
    '50_percentile': group_1.quantile(0.50),
    '75_percentile': group_1.quantile(0.75),
    'iqr': group_1.quantile(0.75) - group_1.quantile(0.25),
    'mode': group_1.mode().iloc[0],  # Pega o primeiro valor da moda
    'skewness': group_1.skew(),
    'kurtosis': group_1.kurt(),
    'sample_size': len(group_1)
}

# Criar DataFrames para exibir as estatísticas
df_stats_0 = pd.DataFrame(stats_0, index=['Fechado = 0'])
df_stats_1 = pd.DataFrame(stats_1, index=['Fechado = 1'])

# Concatenar os DataFrames
df_stats_debt_equity_ratio = pd.concat([df_stats_0, df_stats_1])

df_stats_debt_equity_ratio

A razão dívida/patrimônio líquido tem uma grande variação entre as empresas. Há uma presença significativa de valores negativos.

### Giro de Ativos

O Giro de Ativos mede a eficiência com que a empresa utiliza seus ativos para gerar vendas. Um giro de ativos baixo pode indicar ineficiência na utilização dos ativos. Empresas que não utilizam eficientemente seus ativos podem enfrentar dificuldades de crescimento e rentabilidade, o que pode influenciar negativamente sua viabilidade a longo prazo.

In [ ]:
# Criando o boxplot da margem de lucro, separado por 'fechado'
sns.boxplot(x=data_2012['fechado'], y=np.log1p(data_2012["asset_turnover"]))

# Definindo o título e os labels dos eixos
plt.title('Giro de Ativos vs Fechado')
plt.xlabel('Fechado')
plt.ylabel('Log Giro de Ativos')

# Mostrando o gráfico
plt.show()

A razão dívida/patrimônio líquido tem uma grande variação entre as empresas. Há uma presença significativa de valores negativos, que podem ser outliers ou indicar uma situação financeira atípica. 

In [ ]:
# Separando os dados por grupo para asset_turnover
group_0 = data_2012["asset_turnover"][data_2012['fechado'] == 0]
group_1 = data_2012["asset_turnover"][data_2012['fechado'] == 1]

# Calculando as estatísticas
stats_0 = {
    'mean': group_0.mean(),
    'median': group_0.median(),
    'std_dev': group_0.std(),
    'min': group_0.min(),
    'max': group_0.max(),
    '25_percentile': group_0.quantile(0.25),
    '50_percentile': group_0.quantile(0.50),
    '75_percentile': group_0.quantile(0.75),
    'iqr': group_0.quantile(0.75) - group_0.quantile(0.25),
    'mode': group_0.mode().iloc[0],  # Pega o primeiro valor da moda
    'skewness': group_0.skew(),
    'kurtosis': group_0.kurt(),
    'sample_size': len(group_0)
}

stats_1 = {
    'mean': group_1.mean(),
    'median': group_1.median(),
    'std_dev': group_1.std(),
    'min': group_1.min(),
    'max': group_1.max(),
    '25_percentile': group_1.quantile(0.25),
    '50_percentile': group_1.quantile(0.50),
    '75_percentile': group_1.quantile(0.75),
    'iqr': group_1.quantile(0.75) - group_1.quantile(0.25),
    'mode': group_1.mode().iloc[0],  # Pega o primeiro valor da moda
    'skewness': group_1.skew(),
    'kurtosis': group_1.kurt(),
    'sample_size': len(group_1)
}

# Criando DataFrames para exibir as estatísticas
df_stats_0_asset_turnover = pd.DataFrame(stats_0, index=['Fechado = 0'])
df_stats_1_asset_turnover = pd.DataFrame(stats_1, index=['Fechado = 1'])

# Concatenando os DataFrames
df_stats_asset_turnover = pd.concat([df_stats_0_asset_turnover, df_stats_1_asset_turnover])

df_stats_asset_turnover


Os dados sugerem que, embora a média do giro de ativos seja mais alta para empresas que fecharam, a mediana é semelhante entre os dois grupos. Além disso, a variabilidade no giro de ativos é significativamente maior para empresas que fecharam. A presença de valores extremos é evidente em ambos os grupos, mas é mais pronunciada para empresas que não fecharam.

É importante enfatizar que as análises apresentadas não determinam definitivamente quais variáveis devem ser incluídas no modelo. As quatro variáveis criadas servirão como opções a serem consideradas em diferentes modelos, e sua relevância será avaliada individualmente para cada um deles.

## Limpeza de variáveis provisórias
As variáveis "sales_x1" e "sales_x2" foram usadas na criação da variável dependente mas não são apropriadas para entrarem no modelo (constituiriam um data leak).
Como filtramos pelo ano de 2012, tambem não usaremos a variável "year" na modelagem.

In [ ]:
data_2012 = data_2012.drop(columns=['sales_x1','sales_x2', 'year', 'nace_main', 'comp_id', 'sales', 'balsheet_length'])

#data_2012 = data_2012.drop(columns=['sales_x1','sales_x2', 'year'])

## Matriz de correlação

In [ ]:
# Considera apenas colunas numéricas
numeric_cols = [col for col in data_2012.columns if data_2012[col].dtype in ['int64', 'float64']]

# Remover 'comp_id' da lista de colunas numéricas
if 'comp_id' in numeric_cols:
    numeric_cols.remove('comp_id')

# Lista todas as colunas numéricas exceto "dependente"
cols = [col for col in numeric_cols if col != 'fechado']

# Dividi as colunas em dois grupos
half = len(cols) // 2
group1 = cols[:half] + ['fechado']
group2 = cols[half:] + ['fechado']

# Cria as duas matrizes de correlação
correlation_matrix1 = data_2012[group1].corr()
correlation_matrix2 = data_2012[group2].corr()

# Visualiza a primeira matriz de correlação
plt.figure(figsize=(16,10))
sns.heatmap(correlation_matrix1, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix - Group 1')
plt.show()

# Visualiza a segunda matriz de correlação
plt.figure(figsize=(16,10))
sns.heatmap(correlation_matrix2, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix - Group 2')
plt.show()

In [ ]:
missing_data_6 = show_missing(data_2012).sort_values("pc_missing", ascending=False, ignore_index = True)

missing_data_6.sort_values(by="variable", ignore_index=True)

Confirmamos que o banco de dados final gerado está completo, sem dados faltando através do processo de limpeza com alterações e remoções.

### Análise por agrupamento de variável dependente:
---

Vamos agrupar as empresas marcadas pela variável dependente (fechada) para buscar visualizar algumas das tendências das empresas que fecharam no ano de 2012:

In [ ]:
# Filtrando apenas colunas numéricas
numeric_data_2012 = data_2012.select_dtypes(include=[np.number])

# Agrupando pelo campo "fechado"
groups_2012 = numeric_data_2012.groupby(by="fechado")

# Realizando agregação
df_groups = groups_2012.agg(['mean', 'median'])

df_groups

In [ ]:
sns.boxplot(data=data_2012, x='fechado', y='company_age', 
            showfliers = False, medianprops = {"color":"red"})

# Configuração do gráfico:
plt.xlabel('Status de Operação (fechada)')
plt.ylabel('Idade da empresa')
plt.title('Idade de empresas fechadas vs abertas em 2012')

# Show the plot
plt.show()

print("valores médios de idade da empresa:")
print(df_groups["company_age"]['mean'])

In [ ]:
def group_box(column, data = data_2012):
    ''' Funcao para rodar boxplot agrupado '''
    sns.boxplot(data=data, x='fechado', y=column, 
            showfliers = False, medianprops = {"color":"red"})

    # Configuração do gráfico:
    plt.xlabel('Status de Operação (fechada)')
    plt.ylabel(f'Valor variável "{column}"')
    plt.title('Análise agrupada {column}')

    # Show the plot
    plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

# Create histograms for each group using Seaborn
sns.histplot(data=data_2012, x='company_age', hue='fechado', bins=20, alpha=0.5, ax=ax, kde=True)

# Set labels and title
ax.set_xlabel('Idade da empresa')
ax.set_ylabel('Frequencia da Idade')
ax.set_title('Histograma da Idade das empresas')

# Show the plot
plt.show()

In [ ]:
def group_hist(column, data=data_2012):
    ''' Funcao para gerar o histograma com linha de tendencia agrupado'''
    
    fig, ax = plt.subplots(figsize=(10, 6))

    # Create histograms for each group using Seaborn
    sns.histplot(data=data, x=column, hue='fechado', bins=20, alpha=0.5, ax=ax, kde=True)

    # Set labels and title
    ax.set_xlabel(f'valor da variável {column}')
    ax.set_ylabel(f'Frequência da variável {column}')
    ax.set_title(f'Histograma de {column}')

    # Show the plot
    plt.show()

O grupo de empresas abertas mostra cauda à direita mais substancial que as outras, indicando uma menor tendencia de empresas mais velhas fecharem relativo ao mesmo risco para empresas mais novas.

#### Amortização

In [ ]:
group_box("amort")
group_hist("amort")

Vamos converter para log para conseguirmos analisar melhor as tendências

In [ ]:
criar_log(data_2012, "amort")

group_box("log_amort")
group_hist("log_amort")

O logarítmo da amortização tem distribuição aparentemente normal com cauda à esquerda tanto para as empresas abertas quanto para as fechadas. A distribuição para as empresas abertas tem centro maior do que as outras, sugerindo que um maior valor para o logarítmo da amortização pode indicar maior probabilidade de a empresa estar aberta.

Tendo confirmado uma distribuição mais razoavel para o logarítmo de amortizacção do que para a variável original, tiramos "amort" da base e deixamos "log_amort".

In [ ]:
data_2012.drop(columns = "amort", inplace=True)

In [ ]:
group_hist("ind")

A proporção de empresas fechadas parece se manter por indústria. Dadas as diferenças de escala na quantidade de empresas por indústria, conferimos essas proporções numericamente.

In [ ]:
def proportion_1(column, data=data_2012):
    '''Funcao retorna proporcao de fechado para cada caso'''

    industry_effect = data_2012.groupby(["fechado", column])[column].count()

    pivot_table = data_2012.pivot_table(index=column, columns='fechado', aggfunc='size', fill_value=0)

    # calcular proporção
    pivot_table['Proportion'] = pivot_table[1] / (pivot_table[0] + pivot_table[1])

    # Reset the index to make 'ind' a regular column
    # Resetando index para fazer "ind" uma coluna normal
    pivot_table = pivot_table.reset_index()

    # Renomeando por clareza
    pivot_table.columns = [column, '0_Count', '1_Count', 'Proporcao_1']

    return pivot_table

In [ ]:
print(proportion_1("ind"))

A analise dos números confirma proporcoes aproximadamente iguais de empresas fechadas para abertas por indústria, indicando baixo poder de previsão dessa variável. Por isso retiramos da base as duas variáveis referentes à industria.

In [ ]:
data_2012.drop(columns = ["ind","ind2"], inplace=True)

Agora vamos ordenar por significancia nossas variáveis e verificar os resultados:

Variáveis como "comp_id" número da impressa não deveriam interferir na predição pois são valores arbitrarios de registro de cada empresa.

## Exportação da base tratada para modelagem

In [ ]:
data_2012.to_csv("data_2012.csv", index=False)

In [ ]:
data_2012.columns